In [1]:
import os
import json
import pandas as pd

In [2]:
df = None
for git_name in os.listdir('docs'):
    if '.DS_Store' not in git_name:
        if len(os.listdir(f"docs/{git_name}")) > 0:
            fp = f"docs/{git_name}/objects.json"
            if df is None:
                df = pd.read_json(fp)
            else:
                df_new = pd.read_json(fp)
                df = pd.concat([df, df_new])

In [3]:
df.shape

(74455, 7)

In [4]:
df.head()

,code_before,code_after,commit_msg,com_py2imports_before,com_py2imports_after,diff,com_hash
0,"def _misc(self, func, opts, args):\n ...","def _misc(self, func, opts, args):\n ...",http://code.google.com/p/pytyrant/issues/detai...,"{'0_pytyrant.py': ['import socket', 'import st...","{'0_pytyrant.py': ['import socket', 'import st...","@@ -493,8 +493,10 @@ class Tyrant(object):\n ...",32618ec0a7a05bad587064e20adfb80cb7bd1860
1,"def _tDouble(code, key, integ, fract):\n re...","def _tDouble(code, key, integ, fract):\n re...",fix adddouble typo,"{'0_pytyrant.py': ['import math', 'import sock...","{'0_pytyrant.py': ['import math', 'import sock...","@@ -147,7 +147,7 @@ def _t3F(code, func, opts,...",8b5d9f765a0fbbd76cc7b7ff8d2593436c6a2360
2,def sockstrpair(sock):\n klen = socklen(soc...,def sockstrpair(sock):\n klen = socklen(soc...,change the networking code to use recv and sen...,"{'0_pytyrant.py': ['import socket', 'import st...","{'0_pytyrant.py': ['import socket', 'import st...","@@ -134,27 +134,33 @@ def _t3F(code, func, opt...",22f98fb5400b6234f45fac4bda3ced8b17e7aeb1
3,def socklong(sock):\n return struct.unpack(...,def socklong(sock):\n return struct.unpack(...,change the networking code to use recv and sen...,"{'0_pytyrant.py': ['import socket', 'import st...","{'0_pytyrant.py': ['import socket', 'import st...","@@ -134,27 +134,33 @@ def _t3F(code, func, opt...",22f98fb5400b6234f45fac4bda3ced8b17e7aeb1
4,def socklen(sock):\n return struct.unpack('...,def socklen(sock):\n return struct.unpack('...,change the networking code to use recv and sen...,"{'0_pytyrant.py': ['import socket', 'import st...","{'0_pytyrant.py': ['import socket', 'import st...","@@ -134,27 +134,33 @@ def _t3F(code, func, opt...",22f98fb5400b6234f45fac4bda3ced8b17e7aeb1


In [5]:
df_good = df[df['code_before'] != df['code_after']]

In [6]:
df_good.shape

(69123, 7)

In [7]:
idx=190
df_good.iloc[idx]['com_py2imports_before']

{'0_base.py': ['import copy',
  'from twisted.internet import protocol, reactor, task, defer',
  'from twisted.python import log',
  'from pyamf.util import BufferedByteStream',
  'from fmspy.rtmp.assembly import RTMPAssembler, RTMPDisassembler',
  'from fmspy.rtmp import constants',
  'from fmspy.rtmp.packets import Ping, BytesRead, Invoke',
  'from fmspy.rtmp.status import Status',
  'from fmspy.config import config',
  'from fmspy import _time']}

In [8]:
df_good.iloc[idx]['com_py2imports_after']

{'0_base.py': ['import copy',
  'from twisted.internet import protocol, reactor, task, defer',
  'from twisted.python import log',
  'from pyamf.util import BufferedByteStream',
  'from fmspy.rtmp.assembly import RTMPAssembler, RTMPDisassembler',
  'from fmspy.rtmp import constants',
  'from fmspy.rtmp.packets import Ping, BytesRead, Invoke',
  'from fmspy.rtmp.header import RTMPHeader',
  'from fmspy.rtmp.status import Status',
  'from fmspy.config import config',
  'from fmspy import _time']}

In [9]:
def get_import_diff(row):
    b_k = list(row['com_py2imports_before'])[0] if len(row['com_py2imports_before']) > 0 else ''
    a_k = list(row['com_py2imports_after'])[0] if len(row['com_py2imports_after']) > 0 else ''
    
    if b_k in row['com_py2imports_before']:
        b = set(row['com_py2imports_before'][b_k])
    else:
        b = set()
    
    if a_k in row['com_py2imports_after']:
        a = set(row['com_py2imports_after'][a_k])
    else:
        a = set()
        
    diff_libs = a.symmetric_difference(b)
    return list(diff_libs)

In [10]:
idx = 100

def get_import_union(row):
    b_k = list(row['com_py2imports_before'])[0] if len(row['com_py2imports_before']) > 0 else ''
    a_k = list(row['com_py2imports_after'])[0] if len(row['com_py2imports_after']) > 0 else ''
    
    if b_k in row['com_py2imports_before']:
        b = set(row['com_py2imports_before'][b_k])
    else:
        b = set()
    
    if a_k in row['com_py2imports_after']:
        a = set(row['com_py2imports_after'][a_k])
    else:
        a = set()
        
    diff_libs = a.union(b)
    return list(diff_libs)

df['imports'] = df.apply(lambda x: get_import_union(x), axis=1)

In [11]:
df['imports'].iloc[100]

['from utils import sp_create, sp_create_data',
 'import numpy as np',
 'import numpy.linalg as la',
 'from markovdp import MDP,FastMDP,SparseMDP, Features',
 'import random as pr',
 'import os, sys, getopt, pdb, string']

In [12]:
def lib8funcs(imports):
    lib2funcs = {}
    func2libs = {}
    for imp_line in imports:
        imp_line = imp_line.split('#')[0]
        if imp_line[:4] == 'from':
            try:
                lib, funcs =  imp_line[4:].split(' import ')
                lib = lib.strip()
                funcs = funcs.strip()
                for func in funcs.split(','):
                    func = func.strip()
                    if func[0] == '(':
                        func = func[1:]
                    if func[-1] == ')':
                        func = func[:-1]
                    
                    if func != '*':
                        func2libs[func] = lib.strip()
                    lib2funcs[lib] = func.strip()
                    
            except:
                continue
        else:
            libs = imp_line.split('import')[1]
            libs = libs.strip()
            for lib in libs.split(','): 
                lib = lib.split('as')[0].strip()
                func = '*'
                lib2funcs[lib] = func.strip()
    return lib2funcs, func2libs

In [13]:
df['lib8funcs'] = df['imports'].apply(lambda x: lib8funcs(x))

In [14]:
df['lib8funcs'].iloc[9]

({'struct': '*',
  'itertools': '*',
  'UserDict': '*',
  'math': '*',
  'socket': '*'},
 {})

In [15]:
from collections import Counter

a = Counter()
for row_idx, row in df.iterrows():
    libs = list(row['lib8funcs'][0])
    for el in libs:
        a[el] += 1

In [16]:
from subprocess import call as call_subprocess
import importlib

GOOD_LIBS = set([
    'os', 'sys', 're', 'time', 'logging', 'json', 'urllib', 'datetime', 'urllib3',
    'subprocess', 'random', 'socket', 'threading', '__future__', 'argparse', 'collections',
    'struct', 'shutil', 'string', 'django', 'math', 'traceback', 'optparse', 'sublime',
    'tempfile', 'requests', 'copy', 'itertools', 'getopt', 'twisted', 'signal', 'pickle',
    'unittest', 'simplejson', 'csv', 'functools', 'inspect', 'glob', 'subprocess', 
    'codecs', 'sqlite3', 'types', 'uuid', 'ConfigParser', 'StringIO', 'google', 'pylab',
    'errno', 'pprint', 'gzip', 'gobject', 'BeautifulSoup', 'operator', 'Queue', 'zipfile',
    'httplib3', 'xmlrpclib',  'kivy', 'oauth', 'pynotify', 'SocketServer', 'vim', 'selenium',
    'plistlib', 'pytz', 'theano', 'marshal', 'ssl', 'htmlentitydefs', 'array', 'difflib', 'bottle',
    'cmd', 'glob', 'dateutil', 'psycopg2', 'twitter', 'unicodedata', 'pymongo', 'twisted', 'jinja2',
    'feedparser', 'MySQLdb', 'settings', 'tornado', 'io', 'matplotlib', 'PyQt4', 'string', 'Image', 
    'PIL', 'md5', 'bs4', 'multiprocessing', 'warnings', 'webbrowser', 'xlutils', 'email', 'sqlalchemy',
    'decimal', 'mimetypes', 'calendar', 'yaml', 'xml', 'xmlrpclib', 'gobject', 'errno',
    'numpy', 'pandas', 'sklearn', 'keras', 'torch', 'transformers', 'tensorflow', 'selenium', 
           'scipy', 'statsmodels', 'flask', 'django', 'fastapi', 'gensim', 'nltk'
])
def get_doc(func):
    lib, mod = func.split('.')[0], func.split('.')[1]
    if lib not in GOOD_LIBS:
        return ''
    mod_fin = [el for el in mod.split('.') if el != '*']
    mod_fin = '.'.join(mod_fin)
    doc_lib = ''
    try:
        if len(mod_fin) > 0:
            new_module = __import__(lib, fromlist=[mod_fin])
            doc_lib = new_module.__dict__[str(mod)].__doc__
        else:
            new_module = __import__(lib)
            doc_lib = new_module.__doc__
    except:
        try:
            call_subprocess(f'pip install {lib}'.split())
            if len(mod_fin) > 0:
                new_module = __import__(lib, fromlist=[mod_fin])
                doc_lib = new_module.__dict__[str(mod)].__doc__
            else:
                new_module = __import__(lib)
                doc_lib = new_module.__doc__
        except:
            doc_lib = ''
    return doc_lib
    

def extract_libs(old_text, new_text, lib8funcs_tuple):
    old_lines = set(old_text.split('\n'))
    new_lines = set(new_text.split('\n'))
    diff_lines = old_lines.symmetric_difference(new_lines)
    libs, funcs = list(lib8funcs_tuple[0]), list(lib8funcs_tuple[1])
    
    docs = []
    for line in diff_lines:
        for func_name in funcs:
            if func_name in line:
                func_string = lib8funcs_tuple[1][func_name] + '.' + func_name
                new_doc = get_doc(func_string)
                docs.append(new_doc)
        for lib_name in libs:
            if lib_name in line:
                func_string = lib_name + '.' + lib8funcs_tuple[0][lib_name]
                new_doc = get_doc(func_string)
                if new_doc:
                    docs.append(new_doc)
    return docs

In [17]:
get_doc('socket.*')

"This module provides socket operations and some related functions.\nOn Unix, it supports IP (Internet Protocol) and Unix domain sockets.\nOn other systems, it only supports IP. Functions specific for a\nsocket are available as methods of the socket object.\n\nFunctions:\n\nsocket() -- create a new socket object\nsocketpair() -- create a pair of new socket objects [*]\nfromfd() -- create a socket object from an open file descriptor [*]\nsend_fds() -- Send file descriptor to the socket.\nrecv_fds() -- Recieve file descriptors from the socket.\nfromshare() -- create a socket object from data received from socket.share() [*]\ngethostname() -- return the current hostname\ngethostbyname() -- map a hostname to its IP number\ngethostbyaddr() -- map an IP number or hostname to DNS info\ngetservbyname() -- map a service name and a protocol name to a port number\ngetprotobyname() -- map a protocol name (e.g. 'tcp') to a number\nntohs(), ntohl() -- convert 16, 32 bit int from network to host byte

In [18]:
from tqdm.auto import tqdm

docs = []
for row_idx, row in tqdm(df.iterrows(), total=len(df)):
    x = row
    new_doc = extract_libs(x['code_before'], x['code_after'], x['lib8funcs'])
    docs.append(new_doc)

  0%|          | 0/74455 [00:00<?, ?it/s]

  Using cached email-4.0.2.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      ERROR: Can not execute `setup.py` since setuptools is not available in the build environment.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib
ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib
[INFO   ] [Logger      ] Record log in /Users/Anton.Shapkin/.kivy/logs/kivy_23-03-01_2.txt
[INFO   ] [Kivy        ] v2.1.0
[INFO   ] [Kivy        ] Install

[INFO   ] [Image       ] Providers: img_tex, img_imageio, img_dds, img_sdl2, img_pil (img_ffpyplayer ignored)


  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


...

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.2 requires click>=5.1, but you have click 3.3 which is incompatible.
duckduckgo-search 2.8.0 requires click>=8.1.3, but you have click 3.3 which is incompatible.
distributed 2022.7.0 requires click>=6.6, but you have click 3.3 which is incompatible.
cookiecutter 1.7.3 requires click>=7.0, but you have click 3.3 which is incompatible.
conda-verify 3.4.2 requires click>=6.7, but you have click 3.3 which is incompatible.
black 22.6.0 requires click>=8.0.0, but you have click 3.3 which is incompatible.


  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Attempting uninstall: click
    Found existing installation: click 3.3
    Uninstalling click-3.3:
      Successfully uninstalled click-3.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sublime 0.1.0 requires click<4.0,>=3.3, but you have click 8.1.3 which is incompatible.


  Using cached click-3.3-py2.py3-none-any.whl (58 kB)
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.2 requires click>=5.1, but you have click 3.3 which is incompatible.
duckduckgo-search 2.8.0 requires click>=8.1.3, but you have click 3.3 which is incompatible.
distributed 2022.7.0 requires click>=6.6, but you have click 3.3 which is incompatible.
cookiecutter 1.7.3 requires click>=7.0, but you have click 3.3 which is incompatible.
conda-verify 3.4.2 requires click>=6.7, but you have click 3.3 which is incompatible.
black 22.6.0 requires click>=8.0.0, but you have click 3.3 which is incompatible.
ERROR: Could not find a version that satisfies the requirement Queue (from versions: none)
ERROR: No matching distribution found for Queue
ERROR: Could not find a version that satisfies the requirement Queue (from versions: none)
ERROR: No matching distribution found for Queue


In [19]:
df['docs'] = docs

In [20]:
bb = Counter()
for row_idx, row in df.iterrows():
    bb[len(row['docs'])] += 1

In [21]:
bb

Counter({0: 46973,
         2: 9199,
         1: 5839,
         3: 2638,
         5: 1268,
         4: 3388,
         8: 815,
         6: 1371,
         23: 22,
         24: 32,
         14: 145,
         7: 627,
         12: 279,
         9: 410,
         10: 419,
         16: 108,
         11: 201,
         61: 1,
         13: 156,
         19: 43,
         25: 25,
         22: 35,
         15: 102,
         18: 71,
         20: 44,
         27: 10,
         17: 64,
         21: 31,
         26: 23,
         36: 4,
         34: 3,
         28: 15,
         29: 12,
         32: 9,
         37: 4,
         44: 2,
         38: 4,
         33: 7,
         31: 7,
         35: 11,
         52: 1,
         82: 2,
         45: 2,
         40: 5,
         51: 3,
         42: 2,
         72: 3,
         63: 2,
         62: 1,
         30: 5,
         46: 4,
         54: 2,
         39: 2,
         57: 2,
         79: 1,
         77: 1})

In [23]:
df_new = df.reset_index()
df_new.head()

,index,code_before,code_after,commit_msg,com_py2imports_before,com_py2imports_after,diff,com_hash,imports,lib8funcs,docs
0,0,"def _misc(self, func, opts, args):\n ...","def _misc(self, func, opts, args):\n ...",http://code.google.com/p/pytyrant/issues/detai...,"{'0_pytyrant.py': ['import socket', 'import st...","{'0_pytyrant.py': ['import socket', 'import st...","@@ -493,8 +493,10 @@ class Tyrant(object):\n ...",32618ec0a7a05bad587064e20adfb80cb7bd1860,"[import UserDict, import struct, import socket]","({'UserDict': '*', 'struct': '*', 'socket': '*...",[]
1,1,"def _tDouble(code, key, integ, fract):\n re...","def _tDouble(code, key, integ, fract):\n re...",fix adddouble typo,"{'0_pytyrant.py': ['import math', 'import sock...","{'0_pytyrant.py': ['import math', 'import sock...","@@ -147,7 +147,7 @@ def _t3F(code, func, opts,...",8b5d9f765a0fbbd76cc7b7ff8d2593436c6a2360,"[import struct, import UserDict, import math, ...","({'struct': '*', 'UserDict': '*', 'math': '*',...",[Functions to convert between Python values an...
2,2,def sockstrpair(sock):\n klen = socklen(soc...,def sockstrpair(sock):\n klen = socklen(soc...,change the networking code to use recv and sen...,"{'0_pytyrant.py': ['import socket', 'import st...","{'0_pytyrant.py': ['import socket', 'import st...","@@ -134,27 +134,33 @@ def _t3F(code, func, opt...",22f98fb5400b6234f45fac4bda3ced8b17e7aeb1,"[import UserDict, import struct, import socket]","({'UserDict': '*', 'struct': '*', 'socket': '*...",[]
3,3,def socklong(sock):\n return struct.unpack(...,def socklong(sock):\n return struct.unpack(...,change the networking code to use recv and sen...,"{'0_pytyrant.py': ['import socket', 'import st...","{'0_pytyrant.py': ['import socket', 'import st...","@@ -134,27 +134,33 @@ def _t3F(code, func, opt...",22f98fb5400b6234f45fac4bda3ced8b17e7aeb1,"[import UserDict, import struct, import socket]","({'UserDict': '*', 'struct': '*', 'socket': '*...",[Functions to convert between Python values an...
4,4,def socklen(sock):\n return struct.unpack('...,def socklen(sock):\n return struct.unpack('...,change the networking code to use recv and sen...,"{'0_pytyrant.py': ['import socket', 'import st...","{'0_pytyrant.py': ['import socket', 'import st...","@@ -134,27 +134,33 @@ def _t3F(code, func, opt...",22f98fb5400b6234f45fac4bda3ced8b17e7aeb1,"[import UserDict, import struct, import socket]","({'UserDict': '*', 'struct': '*', 'socket': '*...",[Functions to convert between Python values an...


In [24]:
df_new = df_new.drop(columns=['index'])

In [25]:
df_new.to_json(r'final_data.json')

In [26]:
df_new_disk = pd.read_json(r'final_data.json')

In [27]:
df_new_disk.shape, df_new.shape

((74455, 10), (74455, 10))